In [1]:
import ccxt

# Set up Binance API
binance = ccxt.binance()
symbol = "BTC/USDT"


In [2]:
import import_ipynb
from predictor import XGBoostPredictor, RNNPredictor, LSTMPredictor

import numpy as np


importing Jupyter notebook from predictor.ipynb


In [3]:
rnn = RNNPredictor("../models/rnn_close.h5")
lstm = LSTMPredictor("../models/lstm_close.h5")
xgb = XGBoostPredictor("../models/xgb_close.json")


def predict(inputs, model_name, n_to_predict):
    if model_name == "RNN":
        model = rnn
    elif model_name == "LSTM":
        model = lstm
    elif model_name == "XGBoost":
        model = xgb

    return model.predict(inputs, n_to_predict)


In [4]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Output, Input, State

import plotly.graph_objs as go

import dash_bootstrap_components as dbc


C:\Users\Admin\AppData\Local\Temp\ipykernel_19572\3635243919.py:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\Admin\AppData\Local\Temp\ipykernel_19572\3635243919.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [5]:
dbc_css = "https://cdn.jsdelivr.net/gh/AnnMarieW/dash-bootstrap-templates/dbc.min.css"
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP, dbc_css])

app.layout = html.Div(
    [
        html.H4(
            "BTC-USDT Analysis Dashboard",
            className="bg-primary text-white p-2 mb-2 text-center",
        ),
        dbc.Card(
            [
                html.Div(
                    [
                        dbc.Row(
                            [
                                dbc.Col(dbc.Label("Model"), width=2),
                                dbc.Col(
                                    html.Div(
                                        [
                                            dbc.RadioItems(
                                                id="model-name",
                                                className="btn-group",
                                                inputClassName="btn-check",
                                                labelClassName="btn btn-outline-primary",
                                                labelCheckedClassName="active",
                                                options=["LSTM", "RNN", "XGBoost"],
                                                value="LSTM",
                                            ),
                                        ],
                                        className="radio-group",
                                    ),
                                    width="auto",
                                ),
                            ]
                        ),
                        dbc.Row(
                            [
                                dbc.Col(dbc.Label("Features extraction"), width=2),
                                dbc.Col(
                                    dcc.Dropdown(
                                        id="features",
                                        options=[
                                            "Close",
                                            "Price of Change",
                                            # "RSI",
                                            # "Bolling Bands",
                                            # "Moving Average",
                                        ],
                                        value="Close",
                                        multi=True,
                                    ),
                                    width=7,
                                ),
                            ]
                        ),
                    ]
                ),
            ],
            body=True,
            style={
                "margin": "2rem 1rem 0rem",
            },
        ),
        dbc.Card(
            dbc.Tabs([dbc.Tab([dcc.Graph(id="live-graph")], label="Chart")]),
            style={
                "margin": "2rem 1rem 0rem",
            },
        ),
        html.Div(id="remaining-time"),
        dcc.Interval(
            id="interval-component",
            interval=15 * 1000,  # in milliseconds
            n_intervals=0,
        ),
    ]
)


In [6]:
import pandas as pd


@app.callback(
    [Output("live-graph", "figure"), Output("remaining-time", "children")],
    [Input("interval-component", "n_intervals"), Input("model-name", "value")],
)
def update_graph(n, model_name):
    ohlcv = binance.fetch_ohlcv(symbol, "1m", limit=1000)
    df = pd.DataFrame(
        ohlcv, columns=["timestamp", "open", "high", "low", "close", "volume"]
    )
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")

    input_data = df[["timestamp", "close"]]

    n_to_predict = 100

    predictions = predict(input_data[["timestamp", "close"]], model_name, n_to_predict)
    predictions = list(np.concatenate(predictions))

    trace = go.Candlestick(
        x=df["timestamp"],
        open=df["open"],
        high=df["high"],
        low=df["low"],
        close=df["close"],
        name=symbol,
    )

    latest_timestamp = df["timestamp"].max()
    new_timestamp = latest_timestamp + pd.Timedelta(minutes=1)
    df = df.append({"timestamp": new_timestamp}, ignore_index=True)

    trace_predictions = go.Scatter(
        x=df["timestamp"].iloc[-n_to_predict:],
        y=predictions,
        mode="lines",
        name="Predicted Close",
        line=dict(color="orange"),
    )

    rangeselector = {
        "buttons": list(
            [
                dict(count=1, label="1m", step="minute", stepmode="backward"),
                dict(count=2, label="2m", step="minute", stepmode="backward"),
                dict(count=5, label="5m", step="minute", stepmode="backward"),
                dict(count=1, label="1h", step="hour", stepmode="backward"),
                dict(count=2, label="2h", step="hour", stepmode="backward"),
                dict(count=5, label="5h", step="hour", stepmode="backward"),
                dict(step="all")
            ]
        )
    }

    layout = go.Layout(
        title=f"Live OHLCV Chart - {symbol}",
        xaxis=dict(
            title="Time",
            rangeselector=rangeselector,
        ),
        yaxis=dict(title="Price"),
        showlegend=True,
        height=700,
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    )

    return {
        "data": [trace, trace_predictions],
        "layout": layout,
    }, f"input: {model_name}"


# Start the app.

In [7]:
import webbrowser

if __name__ == "__main__":
    # webbrowser.open('http://127.0.0.1:8050/', new=0, autoraise=True)
    app.run_server(debug=False)


[2023-08-22 11:50:39,219] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "c:\Users\Admin\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 403, in _make_request
    self._validate_conn(conn)
  File "c:\Users\Admin\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 1053, in _validate_conn
    conn.connect()
  File "c:\Users\Admin\anaconda3\lib\site-packages\urllib3\connection.py", line 419, in connect
    self.sock = ssl_wrap_socket(
  File "c:\Users\Admin\anaconda3\lib\site-packages\urllib3\util\ssl_.py", line 449, in ssl_wrap_socket
    ssl_sock = _ssl_wrap_socket_impl(
  File "c:\Users\Admin\anaconda3\lib\site-packages\urllib3\util\ssl_.py", line 493, in _ssl_wrap_socket_impl
    return ssl_context.wrap_socket(sock, server_hostname=server_hostname)
  File "c:\Users\Admin\anaconda3\lib\ssl.py", line 500, in wrap_socket
    return self.sslsocket_class._create(
  File "c:\Users\Admin\anaconda3\lib\ssl.py", line 10

[2023-08-22 17:49:35,211] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "c:\Users\Admin\anaconda3\lib\site-packages\urllib3\connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "c:\Users\Admin\anaconda3\lib\site-packages\urllib3\util\connection.py", line 72, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "c:\Users\Admin\anaconda3\lib\socket.py", line 918, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno 11001] getaddrinfo failed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\Admin\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 714, in urlopen
    httplib_response = self._make_request(
  File "c:\Users\Admin\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 403, in _make_request
   